In [7]:
import fitz  # PyMuPDF
import re
import ollama

def read_pdf(file_path):
    """Membaca teks dari file PDF."""
    doc = fitz.open(file_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

def clean_text(text):
    """Membersihkan teks dari karakter khusus, spasi berlebih, dan format yang tidak diperlukan."""
    text = re.sub(r'\n+', '\n', text)  # Menghapus newline berlebih
    text = re.sub(r'[^a-zA-Z0-9.,\n ]+', '', text)  # Menghapus karakter khusus
    text = re.sub(r' +', ' ', text)  # Menghapus spasi berlebih
    return text.strip()

def chunk_text_semantic(text, chunk_size=1000):
    """Membagi teks menjadi chunk lebih kecil dan memproses setiap chunk dengan model Llama."""
    # Membagi teks menjadi chunk lebih kecil berdasarkan ukuran karakter
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    
    # Memproses setiap chunk menggunakan Llama untuk chunking semantik
    semantic_chunks = []
    for chunk in chunks:
        response = ollama.chat(model='llama3.2', messages=[{
            "role": "user", 
            "content": f"Please divide this text into meaningful semantic chunks:\n\n{chunk}"
        }])
        semantic_chunks.append(response['message']['content'])
    
    return semantic_chunks

def save_to_txt(filename, results):
    """Menyimpan hasil ke file .txt."""
    with open(filename, "w", encoding="utf-8") as f:
        for i, res in enumerate(results):
            f.write(f"Chunk {i+1}:\n{res}\n\n")

# Path ke file PDF
pdf_path = "Draft TKO Pengelolaan Akun Aplikasi.pdf"
output_txt = "output_chunks.txt"

# Membaca, membersihkan, dan membagi teks
raw_text = read_pdf(pdf_path)
cleaned_text = clean_text(raw_text)

# Menggunakan Llama 3.2 untuk chunking semantik
semantic_chunks = chunk_text_semantic(cleaned_text)

# Menyimpan output ke file .txt
save_to_txt(output_txt, semantic_chunks)

print(f"Hasil telah disimpan dalam {output_txt}")

Hasil telah disimpan dalam output_chunks.txt
